In [5]:
import numpy as np
import matplotlib.pyplot as plt
import _pickle as pkl
import pandas as pd
import os,sys,glob, h5py

from dlab import reach_align_pipeline as rf
from dlab.generalephys import placeAxesOnGrid, cleanAxes

from dlab.continuous_traces import gaussian_filter1d
from scipy.signal import find_peaks
from scipy.stats import pearsonr, spearmanr, zscore
from itertools import combinations 

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import seaborn as sns
sns.set_style('white')
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'reach_align_pipeline' from 'dlab' (/opt/anaconda3/envs/dlab_rotation/lib/python3.7/site-packages/dlab/__init__.py)

# import ephys and timestamp info

In [3]:
#import dataframe with information for all mice (dailey- should be on your desktop in kimdata named dfexcept28_ypos.json)
data_path = '/Users/kimberlygagnon/Documents/CU/Welle Lab/experiments/daileydata/dfexcept28_ypos.json'
df = pd.read_json(data_path)

ts_path = '/Users/kimberlygagnon/Documents/CU/Welle Lab/experiments/daileydata/finalized/df_ts.json'
df_ts = pd.read_json(data_path)

p_start = '/Users/kimberlygagnon/Documents/CU/Welle Lab/experiments/daileydata/finalized/Neuropixels_ProcessorStartTimes.csv'
df_start = pd.read_csv(p_start)

# initialize dataframe with behavior alignments

In [ ]:
df_align = pd.DataFrame(df,columns = ['times','ypos','mouse','cohort','cell','overall_rate','baseline_fr','reach_fr','waveform_class'])

# import curated behavior file

In [ ]:
#import curated behavior for specific mouse (save xls as csv)
reach_path = '/Users/kimberlygagnon/Documents/CU/Welle Lab/experiments/daileydata/finalized/Curated_Video_Timestamps/20200804_unit00_session001_NPCE_026.csv'
df_reaches = pd.read_csv(reach_path)
#remove random nans
df_reaches = df_reaches.dropna(axis = 1 , how = 'all')
df_reaches = df_reaches.dropna(axis = 0 , how = 'all')
df_reaches.head()

# synch behavior and create relevant dataframes

In [ ]:
for mouse in df.mouse.unique():
    df_ = df[df.mouse==mouse]
        #align software with ephys
        soft_start = df_start[df_start.Mouse_number==mouse].Processor_Start_time
        frame_ts = df_ts[df_ts.mouse==mouse].timestamps
    #WRITE CODE THAT LOADS RELEVANT CURATED BEHAVIOR EXCEL SPREADSHEET SAVED AS CSV
        reach_times = synchphys(soft_start,frame_ts) #gets reach times
        df_reaches['rMax_t']= reach_times
        df_reaches = df_reaches[df_reaches.behaviors!='none'][df_reaches.behaviors!='arm_movement']
    #in below code can look at success vs. failure reaches by inputing df_reaches[df_reaches=='success'] or 'x_failure'
        reaches = rf.epochfr(df_,df_reaches)
        df_align['epoch_fr'] = reaches
        baseline = rf.epochfr(df_,df_reaches,start = 1.0,end = -0.5,binsize) 
        df_align['baseline_fr'] = baseline
        ave_reach,normedbins,ave_reach_ = rf.reachave_tensor(df_,df_reaches) #gets -4 to +2 epoch for plotting heatmap as well as tensor for PCA
        df_align['bin_ave'] = ave_reach
        df_align['norm_bin_ave'] = normedbins
        traj = rf.center(reach_ave_)
        p,ev = rf.pca(traj)
        df_align['expl_var'] = ev
        df_align['pcs'] = p
        smooth_pcs = gaussian_filter1d(p, sigma=8)
        df_align['smooth_pcs'] = smooth_pcs
        
        #need to concat df_align with larger dataframe in for loop?? to make complete 
    